In [1]:
import pandas as pd
import pickle
import plotly.graph_objs as go
import latextable
from texttable import Texttable
from strategy.strategy import (
    BuyAndHoldStrategy,
    MACDStrategy,
    RSIStrategy,
    ModelQuantilePredictionsStrategy,
    ModelGmadlPredictionsStrategy,
    ConcatenatedStrategies
)
from strategy.util import (
    get_data_windows,
    get_sweep_window_predictions,
    get_predictions_dataframe
)
from strategy.evaluation import (
    parameter_sweep,
    evaluate_strategy
)
from strategy.plotting import (
    plot_sweep_results
)

PADDING=5000
VALID_PART=0.2
INTERVAL='min'
METRIC='mod_ir'
TOP_N=10

In [2]:
data_windows = get_data_windows(
    'wne-masters-thesis-testing',
    'btc-usdt-1m:latest',
    min_window=0, 
    max_window=5
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Downloading large artifact btc-usdt-1m:latest, 3717.80MB. 12 files... 
wandb:   12 of 12 files downloaded.  
Done. 0:0:4.7


In [3]:
def sweeps_on_all_windows(data_windows, strategy_class, params, **kwargs):
    result = []
    for in_sample, _ in data_windows:
        data_part = int((1 - VALID_PART) * len(in_sample))
        result.append(parameter_sweep(in_sample[data_part-PADDING:], strategy_class, params, padding=PADDING, interval=INTERVAL, **kwargs))
    return result

In [4]:
buyandhold_best_strategies = [BuyAndHoldStrategy() for _ in data_windows] 

In [5]:
# Model with gmadl loss
SWEEP_ID = 'filipstefaniuk/wne-masters-thesis-testing/s8goxcbz'
# SWEEP_ID = 'filipstefaniuk/wne-masters-thesis-testing/v3epl3qk'
# train_gmadl_pred_windows = get_sweep_window_predictions(SWEEP_ID, 'train')
valid_gmadl_pred_windows = get_sweep_window_predictions(SWEEP_ID, 'valid')
test_gmadl_pred_windows = get_sweep_window_predictions(SWEEP_ID, 'test')

wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  


In [8]:
# y = test_gmadl_pred_windows[0][2][:, 0, 0]
# fig = go.Figure([
#         go.Scatter(y=y[::100]),
# ])
# fig.show()

In [11]:
MODEL_GMADL_LOSS_FILTER = lambda p: (
    ((p['enter_long'] is not None and (p['enter_short'] is not None or p['exit_long'] is not None))
    or (p['enter_short'] is not None and (p['exit_short'] is not None or p['enter_long'] is not None)))
    and (p['enter_short'] is None or p['exit_long'] is None or (p['exit_long'] > p['enter_short']))
    and (p['enter_long'] is None or p['exit_short'] is None or (p['exit_short'] < p['enter_long'])))

gmadl_model_sweep_results = []
for (in_sample, _), valid_preds, test_preds in zip(data_windows, valid_gmadl_pred_windows, test_gmadl_pred_windows):
    data_part = int((1 - VALID_PART) * len(in_sample))
    params={
        'predictions': [get_predictions_dataframe(valid_preds, test_preds)],
        'enter_long': [None, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007],
        'exit_long': [None, -0.001, -0.002, -0.003, -0.004, -0.005, -0.006, -0.007],
        'enter_short': [None, -0.001, -0.002, -0.003, -0.004, -0.005, -0.006, -0.007],
        'exit_short': [None, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007],
    }
    
    gmadl_model_sweep_results.append(parameter_sweep(
        in_sample[data_part-PADDING:],
        ModelGmadlPredictionsStrategy,
        params,
        params_filter=MODEL_GMADL_LOSS_FILTER,
        padding=PADDING,
        interval=INTERVAL,
        sort_by=METRIC))
    

gmadl_model_best_strategies = [[strat for _, strat in results[:TOP_N]] for results in gmadl_model_sweep_results]

100%|██████████| 1176/1176 [04:30<00:00,  4.35it/s]


In [3]:
# Persist best strategies, so that they don't have to be recomputed every time
best_strategies = {
    'buy_and_hold': buyandhold_best_strategies,
    'gmadl_model': gmadl_model_best_strategies
}

with open('cache/1min-best-strategies-v1.pkl', 'wb') as outp:
    pickle.dump(best_strategies, outp, pickle.HIGHEST_PROTOCOL)

NameError: name 'buyandhold_best_strategies' is not defined

In [4]:
with open('cache/1min-best-strategies-v1.pkl', 'rb') as inpt:
    best_strategies = pickle.load(inpt)

In [3]:
# plot_sweep_results(pd.DataFrame([result for result, _ in gmadl_model_sweep_results[0]]), parameters=['enter_long', 'exit_long', 'enter_short', 'exit_short'], round=5, objective='mod_ir')

In [5]:
def results_plot(idx, result_buyandhold, result_gmadl_model, width=850, height=500, notitle=False, v_lines=None):

    fig = go.Figure([
        go.Scatter(y=result_buyandhold['portfolio_value'], x=result_buyandhold['time'], name="Buy and Hold"),
        go.Scatter(y=result_gmadl_model['portfolio_value'], x=result_gmadl_model['time'], name='GMADL Informer Strategy')
        ])
    
    if v_lines:
        for v_line in v_lines:
            fig.add_shape(
                go.layout.Shape(type="line",
                                yref="paper",
                                xref="x",
                                x0=v_line,
                                x1=v_line,
                                y0=0,
                                y1=1,
                                line=dict(dash='dash', color='rgb(140,140,140)')))
    fig.update_layout(
        title={
        'text': f"W{idx}-{INTERVAL}",
        'y':0.97,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'} if not notitle else None,
        yaxis_title="Portfolio Value",
        xaxis_title="Date",
            font=dict(
            # family="Courier New, monospace",
            size=14,
        ),
        autosize=False,
        width=width,
        height=height,
        margin=dict(l=20, r=20, t=20 if notitle else 110, b=20),
        plot_bgcolor='white',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="left",
            x=0.02
        )
    )
    fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
    )
    fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
    )
    fig.write_image(f"images/eval-w{idx}-{INTERVAL}.png")
    fig.show()
    
def results_table(result_buyandhold, result_gmadl_model):
    table_eval_windows = Texttable()
    table_eval_windows.set_deco(Texttable.HEADER)
    table_eval_windows.set_cols_align(["l", "c","c", "c", "c", "c", "c", "c", "c", "c"])
    table_eval_windows.set_precision(3)

    table_eval_windows.header([
        "\\textbf{Strategy}",
        "\\textbf{VAL}",
        "\\textbf{ARC}",
        "\\textbf{ASD}",
        "\\textbf{IR*}",
        "\\textbf{MD}",
        "\\textbf{IR**}",
        "\\textbf{N}",
        "\\textbf{LONG}",
        "\\textbf{SHORT}",
    ])

    strategy_name_result = [
        ('Buy and Hold', result_buyandhold),
        ('GMADL Informer', result_gmadl_model)
    ]
    for strategy_name, result in strategy_name_result:
        table_eval_windows.add_row([
            strategy_name,
            result['value'],
            f"{result['arc']*100:.2f}\%",
            f"{result['asd']*100:.2f}\%",
            result['ir'],
            f"{result['md']*100:.2f}\%",
            result['mod_ir'],
            result['n_trades'],
            f"{result['long_pos']*100:.2f}\%",
            f"{result['short_pos']*100:.2f}\%",
        ])
    print(latextable.draw_latex(table_eval_windows))


In [6]:
for i, (in_sample, out_of_sample) in enumerate(data_windows):
    padded_window = pd.concat([in_sample.iloc[-PADDING:], out_of_sample])
    result_buyandhold = evaluate_strategy(padded_window, best_strategies['buy_and_hold'][i], padding=PADDING, interval=INTERVAL)
    result_gmadl_model = evaluate_strategy(padded_window, [s[0] for s in best_strategies['gmadl_model']][i], padding=PADDING, interval=INTERVAL)

    results_table(result_buyandhold,  result_gmadl_model)
    # results_plot(i+1, result_buyandhold, result_gmadl_model)

\begin{table}
	\begin{center}
		\begin{tabular}{lccccccccc}
			\textbf{Strategy} & \textbf{VAL} & \textbf{ARC} & \textbf{ASD} & \textbf{IR*} & \textbf{MD} & \textbf{IR**} & \textbf{N} & \textbf{LONG} & \textbf{SHORT} \\
			\hline
			Buy and Hold & 0.929 & -13.87\% & 69.66\% & -0.199 & 52.09\% & -0.053 & 2 & 100.00\% & 0.00\% \\
			GMADL Informer & 1.306 & 71.83\% & 69.69\% & 1.031 & 41.57\% & 1.781 & 50 & 7.29\% & 92.71\% \\
		\end{tabular}
	\end{center}
\end{table}
\begin{table}
	\begin{center}
		\begin{tabular}{lccccccccc}
			\textbf{Strategy} & \textbf{VAL} & \textbf{ARC} & \textbf{ASD} & \textbf{IR*} & \textbf{MD} & \textbf{IR**} & \textbf{N} & \textbf{LONG} & \textbf{SHORT} \\
			\hline
			Buy and Hold & 0.549 & -70.35\% & 73.36\% & -0.959 & 63.40\% & -1.064 & 2 & 100.00\% & 0.00\% \\
			GMADL Informer & 1.837 & 243.15\% & 73.38\% & 3.314 & 25.16\% & 32.024 & 186 & 18.19\% & 81.81\% \\
		\end{tabular}
	\end{center}
\end{table}
\begin{table}
	\begin{center}
		\begin{tabular}{lccccc

In [2]:
# test_data = pd.concat([data_windows[0][0][-PADDING:]] + [data_window[1] for data_window in data_windows])
# buy_and_hold_concat = evaluate_strategy(test_data, BuyAndHoldStrategy(), padding=PADDING, interval=INTERVAL)
# gmadl_model_concat = evaluate_strategy(test_data, ConcatenatedStrategies(len(data_windows[0][1]), [s[0] for s in best_strategies['gmadl_model']], padding=PADDING), padding=PADDING, interval=INTERVAL)

# v_lines=[data_window[1]['close_time'].iloc[-1] for data_window in data_windows][:-1]
# results_table(buy_and_hold_concat, gmadl_model_concat)
# results_plot(0, buy_and_hold_concat, gmadl_model_concat, width=1300, height=500, notitle=True)

In [3]:
import plotly.figure_factory as ff

def results_for_strats(
        data_windows, 
        best_strategies,
        top_n=10):
    test_data = pd.concat([data_windows[0][0][-PADDING:]] + [data_window[1] for data_window in data_windows])

    buy_and_hold_concat = evaluate_strategy(test_data, BuyAndHoldStrategy(), padding=PADDING, interval='min')
    gmadl_1min_model_concat = [evaluate_strategy(test_data, ConcatenatedStrategies(len(data_windows[0][1]), [s[x] for s in best_strategies['gmadl_model']], padding=PADDING), padding=PADDING, interval='min') for x in range(top_n)]

    z = list(reversed([
        list(reversed([round(buy_and_hold_concat['mod_ir'], 3)]*top_n)),
        list(reversed([round(x['mod_ir'], 3) for x in gmadl_1min_model_concat])),
    ]))
    x = list(reversed(range(1, top_n+1)))
    y = list(reversed([
        "Buy and Hold",
        "Gmadl Informer (1 min)"
    ]))
    # 'Portland'
    fig = ff.create_annotated_heatmap(z, x=x, y=y, colorscale='thermal', zmid=buy_and_hold_concat['mod_ir'])
    fig.update_layout(
        margin=dict(l=20, r=20, b=20, t=20),
        width=1100,
        height=650,
        font=dict(
        #    family="Courier New, monospace",
            size=16,  # Set the font size here
            # color="RebeccaPurple"
        )
    )
    fig.show()

# results_for_strats(data_windows, best_strategies, top_n=10) 